In [1]:
import lettuce as lt
import matplotlib.pyplot as plt
import numpy as np
import torch
import os
import csv
from argparse import ArgumentParser, ArgumentDefaultsHelpFormatter

In [2]:
parser = ArgumentParser(formatter_class=ArgumentDefaultsHelpFormatter)
parser.add_argument("--vtkdir", type=str, help="name base of vtk files - they are usually in a vtk-subdirectory!", default="./output/")
parser.add_argument("--csvdir", type=str, default="./output/")
parser.add_argument("--resolution", type=int, default=100)
parser.add_argument("--nout", type=int, default=100)
parser.add_argument("--nvtk", type=int, default=100)
parser.add_argument("--tmax", type=int, default=20)
parser.add_argument("--Re", type=int, default=1600)
parser.add_argument("--collision_operator", type=str, default="BGK")
parser.add_argument("--Precision", type=str, default="Single")
parser.add_argument("--Mach", type=float, default=0.01)
args, unknown = parser.parse_known_args()
args = vars(args)
print("ICH FUNKTIONIERE MIT PULLEN")
Re=args["Re"]
basedir = args["vtkdir"]
csvdir = args["csvdir"]
res = args["resolution"]
nout = args["nout"]
nvtk = args["nvtk"]
tmax = args["tmax"]
Precision= args["Precision"]
collision_operator = args["collision_operator"]
Mach = args["Mach"]
print(collision_operator)
print(torch.version.cuda)

ICH FUNKTIONIERE MIT PULLEN
BGK
12.1


In [3]:
print(torch.cuda._parse_visible_devices())
print(Precision)
if Precision == "Single":
    dtype = torch.float32
elif Precision == "Double":
    dtype = torch.float64
elif Precision == "Half":
    dtype = torch.float16
lattice = lt.Lattice(lt.D3Q27, device = "cpu", dtype=dtype)
flow = lt.TaylorGreenVortex3D(resolution=res, reynolds_number=Re, mach_number=Mach, lattice=lattice)

# Konstruktion des Funktionsnamens
function_name = f"{collision_operator}Collision"

# Überprüfung, ob die Funktion existiert, und deren Aufruf
#if hasattr(lt, function_name):
#    collision = getattr(lt, function_name)(lattice, tau=flow.units.relaxation_parameter_lu)
#    print(collision_operator)
#else:
#    print(":(((")
#    raise ValueError(f"Collision type {collision_operator} is not supported.")
if collision_operator == "BGK":
    collision=lt.BGKCollision(lattice, tau=round(flow.units.relaxation_parameter_lu,7))
elif collision_operator == "KBC":
    collision=lt.KBCCollision3D(lattice, tau=flow.units.relaxation_parameter_lu)
#collision = lt.BGKCollision(lattice, tau=flow.units.relaxation_parameter_lu)
#collision=collision_type(lattice, tau=flow.units.relaxation_parameter_lu)
print(round(flow.units.relaxation_parameter_lu,7))
print(flow.units.viscosity_pu)
streaming = lt.StandardStreaming(lattice)
simulation = lt.Simulation(flow=flow, lattice=lattice, collision=collision, streaming=streaming)


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63]
Single
0.5001723
0.000625


In [4]:
interval_vtk=int((flow.units.convert_time_to_lu(tmax))/20)
interval_out=int((flow.units.convert_time_to_lu(tmax))/500)
Energy = lt.IncompressibleKineticEnergy(lattice, flow)
Dissipation_TGV=lt.Dissipation_sij(lattice, flow)
Enstrophy=lt.Enstrophy(lattice, flow)
Energyspectrum= lt.EnergySpectrum2(lattice, flow)
Symmetrie= lt.SymmetryReporter(lattice,flow)
#SymmetrieCoords = lt.SymmetryTopPercentageReporter(lattice,flow)
interval_out=int((flow.units.convert_time_to_lu(tmax))/500)

reporter = lt.ObservableReporter(Energy, interval=interval_out, out=None)
reporter2 = lt.ObservableReporter(Dissipation_TGV,interval = interval_out, out=None)
reporter3= lt.ObservableReporter(Enstrophy, interval=interval_out, out=None)
reporter4=lt.ObservableReporter(Energyspectrum, interval=interval_out, out=None)
reporter5 = lt.ObservableReporter(Symmetrie, interval=interval_out, out=None)
#reporter6 = lt.ObservableReporter(SymmetrieCoords,interval=interval_vtk, out=None)
simulation.reporters.append(reporter)
simulation.reporters.append(reporter2)
simulation.reporters.append(reporter3)
simulation.reporters.append(reporter4)
simulation.reporters.append(reporter5)
#simulation.reporters.append(reporter6)
simulation.reporters.append(lt.VTKReporter(lattice, flow, interval=interval_vtk, filename_base=basedir+"out"))

steps = int(flow.units.convert_time_to_lu(tmax))


#simulation.reporters.append(lt.VTKReporter(lattice, flow, interval=interval_vtk, filename_base=basedir+"out"))

steps     time     IncompressibleKineticEnergy
steps     time     Dissipation_sij
steps     time     Enstrophy
steps     time     EnergySpectrum2
steps     time     SymmetryReporter


/home/ben/anaconda3/envs/lettuce/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392035891/work/aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [ ]:
#simulation.initialize_f_neq
steps = int(flow.units.convert_time_to_lu(tmax))

mlups = simulation.step(num_steps=steps)
print("Performance in MLUPS:", mlups)
print(steps)
print(flow.units.relaxation_parameter_lu)

In [ ]:
energy = np.array(simulation.reporters[0].out)
print(energy)
dissipation = np.array(simulation.reporters[1].out)
enstrophy=np.array(simulation.reporters[2].out)
energy_spectrum= np.array(simulation.reporters[3].out)

sym = np.array(simulation.reporters[4].out) #ÄNDERN ZU 4 WEGEN ENERGIESPEKTRUM
#sym_top = np.array(simulation.reporters[4].out)


dEdt=np.zeros(len(energy[:,2]))
dt=energy[1,1]-energy[0,1]
nmax=len(energy[:,2])
for i in range(len(energy[:,2])):
    if i == 0:
        dEdt[i] = (energy[1,2] - energy[0,2]) / dt
    elif i == 1:
        dEdt[i] = (energy[2,2] - energy[0,2]) / 2/dt
    elif i == 2:
        dEdt[i] = (1/12*energy[0,2] - 2/3*energy[1,2] + 2/3*energy[3,2] - 1/12*energy[4,2]) / dt
    elif i >= (nmax-3):
        if i == (nmax-3):
            dEdt[i] = (1/12*energy[nmax-5,2] - 2/3*energy[nmax-4,2] + 2/3*energy[nmax-2,2] - 1/12*energy[nmax-1,2]) / dt
        elif i == (nmax-2):
            dEdt[i] = (energy[nmax-1,2] - energy[nmax-3,2]) /2/ dt
        elif i == (nmax-1):
            dEdt[i] = (energy[nmax-1,2] - energy[nmax-2,2]) /dt
    elif i < (nmax-3):
        dEdt[i] = (-1/60*energy[i-3,2] + 3/20*energy[i-2,2] - 3/4*energy[i-1,2] + 3/4*energy[i+1,2] - 3/20*energy[i+2,2] + 1/60*energy[i+3,2]) / dt
dissipationEkin=np.array([energy[:,1],-dEdt/(np.pi*2)**3])
dissipationEns=np.array([enstrophy[:,1],flow.units.viscosity_pu/(np.pi*2)**3*enstrophy[:,2]])
dissipationSij=np.array([dissipation[:,1],dissipation[:,2]])

with open(csvdir + 'DissipationEkin.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(dissipationEkin)
with open(csvdir + 'DissipationEnstrophy.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(dissipationEns)
with open(csvdir + 'DissipationSij.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(dissipationSij)
with open(csvdir + 'EnergySpectrum.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(energy_spectrum)
with open(csvdir + 'Symmetrie.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(sym)

#with open(csvdir + 'SymmetrieTopPercentage.csv', 'w', newline='') as file:
#    writer = csv.writer(file)
#    writer.writerows(sym_top)


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import lettuce as lt

steps = int(1000)
mach_numbers = [0.2, 0.05, 0.01]  # Die drei Mach-Zahlen
colors = ["blue", "red", "green"]  # Farben für die Mach-Zahlen
styles = ["o", "d", "x"]
def mse_error(f1, f2):
    return torch.mean(torch.abs(f2 - f1) / (torch.abs(f2) + 1e-10)).item()

def run_simulation(dtype1, dtype2, mach_number, collision_operator):
    print(f"\n🔹 Starte Simulation: Mach {mach_number} ({dtype1} vs {dtype2})")

    lattice1 = lt.Lattice(lt.D3Q27, device="cuda", dtype=dtype1)
    lattice2 = lt.Lattice(lt.D3Q27, device="cuda", dtype=dtype2)
    flow1 = lt.TaylorGreenVortex3D(resolution=res, reynolds_number=Re, mach_number=mach_number, lattice=lattice1)
    flow2 = lt.TaylorGreenVortex3D(resolution=res, reynolds_number=Re, mach_number=mach_number, lattice=lattice2)
    print(flow1.units.convert_time_to_lu(20))
    dt = flow1.units.convert_time_to_pu(1)
    collision1 = lt.BGKCollision(lattice1, tau=flow1.units.relaxation_parameter_lu)
    collision2 = lt.BGKCollision(lattice2, tau=flow2.units.relaxation_parameter_lu)

    streaming1 = lt.StandardStreaming(lattice1)
    streaming2 = lt.StandardStreaming(lattice2)
    simulation1 = lt.Simulation(lattice=lattice1, flow=flow1, collision=collision1, streaming=streaming1)
    simulation2 = lt.Simulation(lattice=lattice2, flow=flow2, collision=collision2, streaming=streaming2)

    mse_values_short = []
    mse_values_full = []
    labels_short = []
    labels_full = []

    # Initialisierung
    f_i1 = simulation1.f.clone()
    f_i2 = simulation2.f.clone()
    simulation1.initialize_f_neq()
    simulation2.initialize_f_neq()
    f_neq1 = simulation1.f.clone()
    f_neq2 = simulation2.f.clone()

    mse_values_short.append(mse_error(f_i1, f_i2))
    labels_short.append("init")
    mse_values_short.append(mse_error(f_neq1, f_neq2))
    labels_short.append("fneq")

    mse_values_full.append(mse_error(f_neq1, f_neq2))
    labels_full.append(0)

    f_current1 = f_neq1.clone()
    f_current2 = f_neq2.clone()

    for i in range(steps):
        f_s1 = streaming1(f_current1)
        f_s2 = streaming2(f_current2)

        if i < 10:
            mse_values_short.append(mse_error(f_s1, f_s2))
            labels_short.append(f"str. (n={i+1})")

        f_c1 = collision1(f_s1)
        f_c2 = collision2(f_s2)

        mse_values_full.append(mse_error(f_c1, f_c2))
        labels_full.append(i + 1)

        if i < 10:
            mse_values_short.append(mse_error(f_c1, f_c2))
            labels_short.append(f"col. (n={i+1})")

        f_current1 = f_c1.clone()
        f_current2 = f_c2.clone()

    return labels_short, mse_values_short, labels_full, mse_values_full

# **Wissenschaftliche Standards für Paper**
plt.rcParams.update({
    "font.size": 8,
    "axes.labelsize": 8,
    "axes.titlesize": 9,
    "xtick.labelsize": 7,
    "ytick.labelsize": 7,
    "legend.fontsize": 7,
    "figure.titlesize": 9
})

fig, axs = plt.subplots(1, 2, figsize=(7, 2.5))
sizes = [6,5,5]
for mach, color, style, markersize in zip(mach_numbers, colors, styles, sizes):
    labels_short, mse_values_short, labels_full, mse_values_full = run_simulation(torch.float32, torch.float64, mach, "BGK")

    # 🔹 **Plot 1: Erste 10 Schritte mit Init & fneq**
    axs[0].plot(range(len(labels_short)), mse_values_short, label=f"Mach {mach}", color=color, marker=style, markersize = markersize)
    axs[0].set_xticks(range(len(labels_short)))
    axs[0].set_xticklabels(labels_short, rotation=90)

    # 🔹 **Plot 2: Alle Schritte**
    axs[1].plot(labels_full, mse_values_full, label=f"Mach {mach}", color=color, marker = style, markevery=100, markersize = markersize)

axs[0].set_ylabel("f MSE")
axs[0].grid()
axs[0].legend()

axs[1].set_xticks([0] + list(range(int(steps/10), steps + 1, int(steps/10))))
axs[1].set_xticklabels([0] + list(range(int(steps/10), steps + 1, int(steps/10))), rotation=90)
axs[1].set_xlabel("Steps")
axs[1].grid()

# **Speichern & Anzeigen**
output_path = '/work/bpicar3s/MP2/TGV_Comparison/' + 'SimStepBGK_MSE_vs_Step'+str(res)+'.pdf'
plt.savefig(output_path, format='pdf', bbox_inches="tight")
plt.tight_layout()
plt.show()


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import lettuce as lt

mach_numbers = [0.2, 0.05, 0.01]  # Die drei Mach-Zahlen
colors = ["red", "blue", "green"]  # Farben für die Mach-Zahlen

def mse_error(f1, f2):
    return torch.mean(torch.abs(f2 - f1) / (torch.abs(f2))).item()

def run_simulation(dtype1, dtype2, mach_number, collision_operator, target_time_pu=20):
    print(f"\n🔹 Starte Simulation: Mach {mach_number} ({dtype1} vs {dtype2})")

    lattice1 = lt.Lattice(lt.D3Q27, device="cuda", dtype=dtype1)
    lattice2 = lt.Lattice(lt.D3Q27, device="cuda", dtype=dtype2)
    flow1 = lt.TaylorGreenVortex3D(resolution=res, reynolds_number=Re, mach_number=mach_number, lattice=lattice1)
    flow2 = lt.TaylorGreenVortex3D(resolution=res, reynolds_number=Re, mach_number=mach_number, lattice=lattice2)

    total_steps =int( flow1.units.convert_time_to_lu(target_time_pu))  # Physikalischer Zeitschritt
    dt = flow1.units.convert_time_to_pu(1) # Anzahl der Schritte für 10s Simulation

    print(f"🔹 Mach {mach_number}: dt = {dt}, benötigte Schritte für {target_time_pu}s: {total_steps}")

    collision1 = lt.BGKCollision(lattice1, tau=flow1.units.relaxation_parameter_lu)
    collision2 = lt.BGKCollision(lattice2, tau=flow2.units.relaxation_parameter_lu)

    streaming1 = lt.StandardStreaming(lattice1)
    streaming2 = lt.StandardStreaming(lattice2)
    simulation1 = lt.Simulation(lattice=lattice1, flow=flow1, collision=collision1, streaming=streaming1)
    simulation2 = lt.Simulation(lattice=lattice2, flow=flow2, collision=collision2, streaming=streaming2)

    mse_values = []
    time_values = []

    # Initialisierung
    f_i1 = simulation1.f.clone()
    f_i2 = simulation2.f.clone()
    simulation1.initialize_f_neq()
    simulation2.initialize_f_neq()
    f_neq1 = simulation1.f.clone()
    f_neq2 = simulation2.f.clone()

    mse_values.append(mse_error(f_neq1, f_neq2))
    time_values.append(0)

    f_current1 = f_neq1.clone()
    f_current2 = f_neq2.clone()

    for i in range(total_steps):
        f_s1 = streaming1(f_current1)
        f_s2 = streaming2(f_current2)

        f_c1 = collision1(f_s1)
        f_c2 = collision2(f_s2)

        mse_values.append(mse_error(f_c1, f_c2))
        time_values.append((i + 1) * dt)  # Physikalische Zeit

        f_current1 = f_c1.clone()
        f_current2 = f_c2.clone()

    return time_values, mse_values

# **Wissenschaftliche Standards für Paper**
plt.rcParams.update({
    "font.size": 8,
    "axes.labelsize": 8,
    "axes.titlesize": 9,
    "xtick.labelsize": 7,
    "ytick.labelsize": 7,
    "legend.fontsize": 7,
    "figure.titlesize": 9
})

fig, ax = plt.subplots(figsize=(3.5, 2.5))

for mach, color in zip(mach_numbers, colors):
    time_values, mse_values = run_simulation(torch.float32, torch.float64, mach, collision_operator)

    # **MSE vs. physikalische Zeit plotten**
    ax.plot(time_values, mse_values, label=f"Mach {mach}", color=color)

ax.set_xlabel("Physical Time (s)")
ax.set_ylabel("f MSE")
ax.grid()
ax.legend()

# **Speichern & Anzeigen**
output_path = '/work/bpicar3s/MP2/TGV_Comparison/' + 'SimStepBGK_MSE_vs_Time'+str(Re)+'.pdf'
plt.savefig(output_path, format='pdf', bbox_inches="tight")
plt.tight_layout()
plt.show()


In [ ]:
Rey = Re

if Rey == 100 or Rey == 800:
    csv_file_path = f'/home/ben/Schreibtisch/Enstropy Brachet/Brachet/Brachet_1983_tgv3d_enstrophy_Re{Rey}.csv'
    csv_data = []
    with open(csv_file_path, 'r') as file:
        csv_reader = csv.reader(file, delimiter=';')
        for row in csv_reader:
            csv_data.append(row)
    csv_data = np.array(csv_data, dtype=float).T
elif Rey == 1600:
    csv_file_path = f"/home/ben/Schreibtisch/Referenz/Re{Rey}.csv"
    csv_data = np.genfromtxt(csv_file_path, delimiter=";", dtype=str)
    csv_data = np.char.replace(csv_data, ',', '.').astype(float).T
else:
    csv_data = np.loadtxt(f"/home/ben/Schreibtisch/Referenz/Re{Rey}.csv", delimiter=",")

plt.plot(csv_data[0,:], csv_data[1,:])
plt.plot(dissipationEkin[0,:], dissipationEkin[1,:])
plt.ylim(0,0.02)

In [ ]:
print(Precision)
if Precision == "Single":
    dtype = torch.float32
elif Precision == "Double":
    dtype = torch.float64
elif Precision == "Half":
    dtype = torch.float16

# **Erste Simulation mit Float32 für Initialisierung**
lattice_32 = lt.Lattice(lt.D3Q27, device="cuda", dtype=torch.float32)
flow_32 = lt.TaylorGreenVortex3D(resolution=res, reynolds_number=Re, mach_number=Mach, lattice=lattice_32)

if collision_operator == "BGK":
    collision_32 = lt.BGKCollision(lattice_32, tau=np.float64(flow_32.units.relaxation_parameter_lu))
elif collision_operator == "KBC":
    collision_32 = lt.KBCCollision3D(lattice_32, tau=np.float64(flow_32.units.relaxation_parameter_lu))

streaming_32 = lt.StandardStreaming(lattice_32)
simulation_32 = lt.Simulation(flow=flow_32, lattice=lattice_32, collision=collision_32, streaming=streaming_32)

# **Initialisierung mit Float32**
simulation_32.initialize_f_neq()

# **Konvertiere `f` auf Float64**
f_64 = simulation_32.f.to(torch.float64)
print(simulation_32.f.dtype)
print(f_64.dtype)

reporter = lt.ObservableReporter(Energy, interval=interval_out, out=None)

simulation.reporters.append(reporter)

steps = int(flow.units.convert_time_to_lu(tmax))


#simulation.reporters.append(lt.VTKReporter(lattice, flow, interval=interval_vtk, filename_base=basedir+"out"))


# **Erstelle neue Simulation mit Float64**
lattice_64 = lt.Lattice(lt.D3Q27, device="cuda", dtype=torch.float64)
flow_64 = lt.TaylorGreenVortex3D(resolution=res, reynolds_number=Re, mach_number=Mach, lattice=lattice_64)

steps = int(flow.units.convert_time_to_lu(tmax))

if collision_operator == "BGK":
    collision_64 = lt.BGKCollision(lattice_64, tau=np.float64(np.float32(flow_32.units.relaxation_parameter_lu)))
elif collision_operator == "KBC":
    collision_64 = lt.KBCCollision3D(lattice_64, tau=np.float64(np.float32(flow_32.units.relaxation_parameter_lu)))

streaming_64 = lt.StandardStreaming(lattice_64)
simulation_64 = lt.Simulation(flow=flow_64, lattice=lattice_64, collision=collision_64, streaming=streaming_64)


Energy = lt.IncompressibleKineticEnergy(lattice=lattice_64, flow=flow_64)

reporter = lt.ObservableReporter(Energy, interval=interval_out, out=None)

simulation_64.reporters.append(reporter)

# **Setze `f` aus der Float32-Initialisierung in die neue Simulation**
simulation_64.f = f_64

# **Starte die Simulation mit Float64**
steps = int(flow_64.units.convert_time_to_lu(tmax))
mlups = simulation_64.step(num_steps=steps)

print("Performance in MLUPS:", mlups)
print(steps)
print(flow_64.units.relaxation_parameter_lu)

In [ ]:
energy = np.array(simulation_64.reporters[0].out)
print(energy)
dEdt=np.zeros(len(energy[:,2]))
dt=energy[1,1]-energy[0,1]
nmax=len(energy[:,2])
for i in range(len(energy[:,2])):
    if i == 0:
        dEdt[i] = (energy[1,2] - energy[0,2]) / dt
    elif i == 1:
        dEdt[i] = (energy[2,2] - energy[0,2]) / 2/dt
    elif i == 2:
        dEdt[i] = (1/12*energy[0,2] - 2/3*energy[1,2] + 2/3*energy[3,2] - 1/12*energy[4,2]) / dt
    elif i >= (nmax-3):
        if i == (nmax-3):
            dEdt[i] = (1/12*energy[nmax-5,2] - 2/3*energy[nmax-4,2] + 2/3*energy[nmax-2,2] - 1/12*energy[nmax-1,2]) / dt
        elif i == (nmax-2):
            dEdt[i] = (energy[nmax-1,2] - energy[nmax-3,2]) /2/ dt
        elif i == (nmax-1):
            dEdt[i] = (energy[nmax-1,2] - energy[nmax-2,2]) /dt
    elif i < (nmax-3):
        dEdt[i] = (-1/60*energy[i-3,2] + 3/20*energy[i-2,2] - 3/4*energy[i-1,2] + 3/4*energy[i+1,2] - 3/20*energy[i+2,2] + 1/60*energy[i+3,2]) / dt
dissipationEkininit=np.array([energy[:,1],-dEdt/(np.pi*2)**3])

In [ ]:
Rey = Re

if Rey == 100 or Rey == 800:
    csv_file_path = f'/home/ben/Schreibtisch/Enstropy Brachet/Brachet/Brachet_1983_tgv3d_enstrophy_Re{Rey}.csv'
    csv_data = []
    with open(csv_file_path, 'r') as file:
        csv_reader = csv.reader(file, delimiter=';')
        for row in csv_reader:
            csv_data.append(row)
    csv_data = np.array(csv_data, dtype=float).T
elif Rey == 1600:
    csv_file_path = f"/home/ben/Schreibtisch/Referenz/Re{Rey}.csv"
    csv_data = np.genfromtxt(csv_file_path, delimiter=";", dtype=str)
    csv_data = np.char.replace(csv_data, ',', '.').astype(float).T
else:
    csv_data = np.loadtxt(f"/home/ben/Schreibtisch/Referenz/Re{Rey}.csv", delimiter=",")

plt.plot(csv_data[0,:], csv_data[1,:], label = 'Ref')
plt.plot(dissipationEkin[0,:], dissipationEkin[1,:], label = 'Stanni')
plt.plot(dissipationEkininit[0,:], dissipationEkininit[1,:], label = 'Init')
plt.legend()
plt.show()

In [ ]:
plt.plot(dissipationEkininit[0,:], (dissipationEkin[1,:]-dissipationEkininit[1,:])/dissipationEkin[1,:])

print(dissipationEkininit[0,0], dissipationEkin[0,0])

In [ ]:
diff = np.float64(np.float32(flow_32.units.relaxation_parameter_lu)) - flow_64.units.relaxation_parameter_lu
print(diff)
print(simulation_32.f.dtype)
print(np.float64(np.float32(flow_32.units.relaxation_parameter_lu)))
print(flow_32.units.relaxation_parameter_lu)

tensor_32 = torch.tensor(1.0, dtype=torch.float32)  # PyTorch float32
numpy_64 = np.float64(2.0)  # NumPy float64

result = tensor_32 + numpy_64  # Addition zwischen torch.float32 und numpy.float64

print(result)
print(result.dtype)  # Ergebnis ist ein numpy.float64-Objekt!


In [ ]:
import numpy as np

# Lange Zahl in Float64
long_number = 0.123456789123456789

# Umwandlung in Float64
num_float64 = np.float64(long_number)

# Umwandlung in Float32
num_float32 = np.float32(num_float64)

# Unterschied berechnen
diff = num_float64 - np.float64(num_float32)

# Ergebnisse in einem Dictionary speichern und anzeigen
results = {
    "Original Float64": num_float64,
    "Converted Float32": num_float32,
    "Difference": diff,
}

print(results)
import pandas as pd

df_results = pd.DataFrame.from_dict(results, orient="index", columns=["Value"])
